In [2]:
import re 
import os
import vrplib
import random
import matplotlib.pyplot as plt

# 准备工作

In [3]:

#按文件名排序加载数据
def read_all_instances(root_folder, ending='.tsp'):  
    instances = []  
    
    def extract_k_number(file_name):  
        match = re.search(r'k(\d+)', file_name)  
        if match:  
            return int(match.group(1))
        return float('inf')  

    file_names = sorted(  
        [file_name for file_name in os.listdir(root_folder) if file_name.endswith(ending)],  
        key=extract_k_number 
    )   
    for file_name in file_names:  
        instance = vrplib.read_instance(str(os.path.join(root_folder, file_name))) 
        if instance:  
            instances.append(instance)  
            print(f'Successfully read {file_name}')  
        else:
            print(f'Failed to read {file_name}')  
    
    return instances  

In [5]:
root_folder = './data/cvrp/new_data/'  
cvrp_instances = read_all_instances(root_folder, ending='.vrp')
for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = cvrp_instance['edge_weight']
    node_coords = cvrp_instance['node_coord']
    #print(cvrp_instance['comment'])
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  


Successfully read A-n34-k5.vrp
Successfully read A-n32-k5.vrp
Successfully read A-n36-k5.vrp
Successfully read A-n33-k5.vrp
Successfully read A-n37-k5.vrp
Successfully read A-n38-k5.vrp
Successfully read A-n39-k5.vrp
Successfully read A-n39-k6.vrp
Successfully read A-n37-k6.vrp
Successfully read A-n33-k6.vrp
Successfully read A-n44-k6.vrp
Successfully read A-n45-k6.vrp
Successfully read A-n53-k7.vrp
Successfully read A-n46-k7.vrp
Successfully read A-n54-k7.vrp
Successfully read A-n45-k7.vrp
Successfully read A-n48-k7.vrp
Successfully read A-n62-k8.vrp
Successfully read A-n65-k9.vrp
Successfully read A-n69-k9.vrp
Successfully read A-n61-k9.vrp
Successfully read A-n64-k9.vrp
Successfully read A-n55-k9.vrp
Successfully read A-n60-k9.vrp
Successfully read A-n63-k9.vrp
Successfully read A-n63-k10.vrp
Successfully read A-n80-k10.vrp


In [11]:
# Function to visualize solution
def plot_CVRP_solution(routes, node_coords):
    plt.figure(figsize=(10, 8))
    
    # Plot nodes
    for i, (x, y) in enumerate(node_coords):
        plt.scatter(x, y, c='blue' if i == 0 else 'red')
        plt.text(x, y, f'{i}', fontsize=9, ha='right')
    
    # Plot routes
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
    for vehicle, route in enumerate(routes):
        route_coords = [node_coords[0]] + [node_coords[node] for node in route] + [node_coords[0]]
        x_coords, y_coords = zip(*route_coords)
        plt.plot(x_coords, y_coords, c=colors[vehicle % len(colors)], label=f'Vehicle {vehicle + 1}')
    
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routing Problem Solution')
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to test vehicle capacity constraint
def test_capacity_constraint(routes, demands, vehicle_capacity):
    for vehicle, route in enumerate(routes):
        total_demand = sum(demands[node] for node in route)
        if total_demand > vehicle_capacity:
            print(f"Vehicle {vehicle + 1} exceeds capacity: {total_demand} > {vehicle_capacity}")
        else:
            print(f"Vehicle {vehicle + 1} is within capacity: {total_demand} <= {vehicle_capacity}")


# Function to calculate route distance
def calculate_route_distance(route, distance_matrix):
    distance = 0
    if route:
        distance += distance_matrix[0][route[0]]  # From depot to first node
        for i in range(1, len(route)):
            distance += distance_matrix[route[i-1]][route[i]]
        distance += distance_matrix[route[-1]][0]  # From last node back to depot
    return distance

# Function to calculate total distance of all routes (CVRP)
def total_distance(routes, distance_matrix):
    return sum(calculate_route_distance(route, distance_matrix) for route in routes)


# Generate initial solution using a greedy approach
def greedy_initial_solution():
    routes = [[] for _ in range(num_vehicles)]
    current_load = [0] * num_vehicles
    visited = [False] * num_nodes
    visited[0] = True  # Starting from the depot

    for vehicle in range(num_vehicles):
        current_node = 0
        while True:
            next_node = None
            min_distance = float('inf')
            for i in range(1, num_nodes):
                if not visited[i] and current_load[vehicle] + demands[i] <= vehicle_capacity:
                    if distance_matrix[current_node][i] < min_distance:
                        min_distance = distance_matrix[current_node][i]
                        next_node = i
            if next_node is None:
                break
            routes[vehicle].append(next_node)
            current_load[vehicle] += demands[next_node]
            visited[next_node] = True
            current_node = next_node
    return routes


# Large Neighborhood Search, LNS

**大邻域搜索（Large Neighborhood Search, LNS）** 优化技术。

算法的基本原理：
初始解生成：

使用贪心算法（greedy_initial_solution）构造初始解，为每辆车分配尽可能多的客户，但同时保证不超过车辆容量。
破坏-修复策略（Destroy and Repair）：

- 破坏（Destroy Phase）：通过destroy_solution函数，随机移除一些客户（例如 destruction_size 个客户）从当前解中，制造一个部分解。
- 修复（Repair Phase）：通过repair_solution函数，将被移除的客户重新插入部分解中，尽量使重新插入的成本最小化，生成一个新的解决方案。
循环优化（Iterative Optimization）：

使用一定次数（例如 1000 次迭代）的破坏-修复操作，尝试寻找更优的路径。
不断比较每次修复后的解与当前最优解，如果发现新的解更优（路线总距离更短），则更新当前解和最优解。
约束检查：

在修复阶段，repair_solution会检查车辆容量约束，保证插入的客户不会导致分配给一个车辆的总需求超过其容量。


算法特点：
- 启发式方法：大邻域搜索是一种启发式算法，目的是探索更大的解空间，跳出局部最优解。
- 灵活性：破坏和修复的过程可以通过参数（例如破坏规模）来调整，以适应不同的问题规模。
- 效率高：在实际应用中，大邻域搜索算法常常具有较好的收敛速度，并且能在合理时间内求得高质量的解。


In [7]:
def destroy_solution(routes, num_customers_to_remove):
    # Flatten routes to get a list of all customers
    all_customers = [customer for route in routes for customer in route]

    # Randomly select customers to remove
    removed_customers = random.sample(all_customers, num_customers_to_remove)

    new_routes = []
    for route in routes:
        new_route = [customer for customer in route if customer not in removed_customers]
        new_routes.append(new_route)
    
    return new_routes, removed_customers


In [8]:
def repair_solution(routes, removed_customers, distance_matrix, demands, vehicle_capacity, num_vehicles):
    for customer in removed_customers:
        best_insertion = None
        best_cost_increase = float('inf')
        best_route_idx = None
        best_insert_position = None

        for route_idx, route in enumerate(routes):
            route_demand = sum(demands[node] for node in route)

            if route_demand + demands[customer] > vehicle_capacity:
                continue # skip this route

            for i in range(len(route) + 1): # +1 you are allowed to insert the customer to the end
                new_route = route[:i] + [customer] + route[i:] # A - B -> Insert C. Cost(A-C-B) -  Cost (A-B)
                cost_increase = calculate_route_distance(new_route, distance_matrix) - calculate_route_distance(route, distance_matrix)
                if cost_increase < best_cost_increase:
                    best_cost_increase = cost_increase
                    best_insertion = new_route
                    best_route_idx = route_idx
                    best_insert_position = i
            
        if best_insertion is not None:
            routes[best_route_idx] = best_insertion
        else:
            if len(routes) < num_vehicles:
                routes.append([customer])
            else:
                routes[0].append(customer)

    return routes

In [9]:
def large_neighborhood_search_cvrp(distance_matrix, demands, num_vehicles, vehicle_capacity, num_iterations = 1000, destruction_size= 5):
    current_routes = greedy_initial_solution()
    current_cost = sum(calculate_route_distance(route, distance_matrix) for route in current_routes)
    best_routes = [route.copy() for route in current_routes]
    best_cost = current_cost

    for iteration in range(num_iterations):

        # Destroy Phase
        partial_routes, removed_customers = destroy_solution(current_routes, destruction_size)

        # Repair Phase
        repaired_routes = repair_solution(partial_routes, removed_customers, distance_matrix, demands, vehicle_capacity, num_vehicles)

        # Calculate the new cost of the repaired solution
        repaired_cost = sum(calculate_route_distance(route, distance_matrix) for route in repaired_routes)

        # Comparision
        if repaired_cost < current_cost:
            current_routes = [route.copy() for route in repaired_routes]
            current_cost = repaired_cost

            if current_cost < best_cost:
                best_routes = [route.copy() for route in current_routes]
                best_cost = current_cost

    return best_routes, best_cost

In [18]:
for i in range(len(cvrp_instances)):
    cvrp_instance = cvrp_instances[i]
    vehicle_capacity = cvrp_instance['capacity']
    demands = cvrp_instance['demand']
    distance_matrix = cvrp_instance['edge_weight']
    node_coords = cvrp_instance['node_coord']  #数据中点的坐标
    print("=====================",cvrp_instance['comment'],"=====================")
    match = re.search(r"No of trucks: (\d+)", str(cvrp_instance['comment']))  
    if match:  
        num_vehicles = int(match.group(1))  
    num_nodes = len(node_coords)
    print("Number of Nodes:",num_nodes)
    best_routes, best_cost = large_neighborhood_search_cvrp(distance_matrix, demands, num_vehicles, vehicle_capacity, num_iterations=1000, destruction_size=5)
    print("Best routes found:", best_routes)
    print("Best total distance:", best_cost)
    test_capacity_constraint(best_routes, demands, vehicle_capacity)
    #plot_CVRP_solution(best_routes, node_coords)

===================== (Augerat et al, No of trucks: 5, Optimal value: 778) =====================
Number of Nodes: 34
Best routes found: [[26, 4, 33, 16, 20], [5, 30, 24, 27, 1, 29], [14, 23, 11, 8, 15, 7, 13, 10], [18, 2, 22, 9, 12, 3, 32, 21], [6, 19, 17, 25, 31, 28]]
Best total distance: 805.4956442169512
Vehicle 1 is within capacity: 73 <= 100
Vehicle 2 is within capacity: 92 <= 100
Vehicle 3 is within capacity: 100 <= 100
Vehicle 4 is within capacity: 98 <= 100
Vehicle 5 is within capacity: 97 <= 100
===================== (Augerat et al, No of trucks: 5, Optimal value: 784) =====================
Number of Nodes: 32
Best routes found: [[30, 16, 1, 12], [24, 27], [21, 31, 19, 17, 13, 7, 26], [6, 3, 2, 23, 4, 11, 28, 14], [20, 5, 25, 10, 29, 15, 22, 9, 8, 18]]
Best total distance: 787.0818888551103
Vehicle 1 is within capacity: 72 <= 100
Vehicle 2 is within capacity: 44 <= 100
Vehicle 3 is within capacity: 98 <= 100
Vehicle 4 is within capacity: 98 <= 100
Vehicle 5 is within capacity: